In [ ]:
# Program: Einstein Solid Model.ipynb
# Purpose: Modeling how the addition of energy quanta affect a two particle (Al + Pb) Einstein Solid
# Author:  Tobias Safie - tks57@drexel.edu
# Date:    February 2, 2025

from vpython import *
from math import *
from random import *


# Constants
N_A = 6.022e23	# [mol^-1] Avagadro's number
hbar = 1.05e-34	# [kg m^2 /s] Planck's reduced constant
kb = 1.38e-23	# [J/K] Boltzmann constant
l = 5e-10	# length of spring [m]  (separation of atoms)
dt = 1e-14		# [s] Time step
t = 0			# [s] Initial time
N = 3			# number of oscillators in each atom

# Create Al atom
Al = sphere(color=color.blue, radius=143e-12, pos=vec(-l,0,0))

# Create Pb atom (as above)
Pb = sphere(color=color.red, radius=1.8e-10, pos=vec(l,0,0))

# Button 1 - Add one quanta to each atom in a random direction
def click1(evt):
    global Al_dq, Pb_dq
    i = randint(0,2)
    Al_dq[i] = 1
    Pb_dq[i] = 1
    
# Button 2 - Add 5 Al quanta to each atom in a random direction
def click2(evt):
    global Al_dE, Pb_dE, Al_dq, Pb_dq
    i = randint(0,2)
    Al_dq[i] = 5
    Pb_dq[i] = 5

Al_k = 2*16.# [N/m] Al effective spring constant
Pb_k = 2*5.# [N/m] Pb effective spring constant

# Define energy of one quantum and mass of each atom
#  Note that I have assigned the mass as a property of the atom
#  rather than a separate variable (Al.m vs Al_m).  That is a matter
#  of taste/style.  Ideally you'd do one or the other and stick with it
#  instead of mixing.
Al_m = 26.98 / (N_A * 1000) #Mass of one Al atom
Al_dE = hbar * sqrt(Al_k / Al_m) #Energy of one Al quantum
Pb_m = 207.2 / (N_A * 1000) #Mass of one Pb atom
Pb_dE = hbar * sqrt(Pb_k / Pb_m) #Energy of one Pb quantum

# Initialize list to store Al's springs
Al_spring = []

# Attach Al springs in all 6 directions
Al_spring.append(helix(pos=vec(0,0,0), axis=vec(-l,0,0), radius=0.5e-10))  # +x direction (Wall at 0, pulling to -l)
Al_spring.append(helix(pos=vec(-2*l,0,0), axis=vec(l,0,0), radius=0.5e-10))  # -x direction (Wall at -2l, pulling to -l)
Al_spring.append(helix(pos=vec(-l,l,0), axis=vec(0,-l,0), radius=0.5e-10))  # +y direction (Wall at -l,l)
Al_spring.append(helix(pos=vec(-l,-l,0), axis=vec(0,l,0), radius=0.5e-10))  # -y direction (Wall at -l,-l)
Al_spring.append(helix(pos=vec(-l,0,l), axis=vec(0,0,-l), radius=0.5e-10))  # +z direction (Wall at -l,0,l)
Al_spring.append(helix(pos=vec(-l,0,-l), axis=vec(0,0,l), radius=0.5e-10))  # -z direction (Wall at -l,0,-l)

# Initialize list to store Pb's springs
Pb_spring = []

# Attach Pb springs in all 6 directions
Pb_spring.append(helix(pos=vec(2*l,0,0), axis=vec(-l,0,0), radius=0.5e-10))  # +x direction (Wall at 2l, pulling to +l)
Pb_spring.append(helix(pos=vec(0,0,0), axis=vec(l,0,0), radius=0.5e-10))  # -x direction (Wall at 0, pulling to +l)
Pb_spring.append(helix(pos=vec(l,l,0), axis=vec(0,-l,0), radius=0.5e-10))  # +y direction (Wall at l,l)
Pb_spring.append(helix(pos=vec(l,-l,0), axis=vec(0,l,0), radius=0.5e-10))  # -y direction (Wall at l,-l)
Pb_spring.append(helix(pos=vec(l,0,l), axis=vec(0,0,-l), radius=0.5e-10))  # +z direction (Wall at l,0,l)
Pb_spring.append(helix(pos=vec(l,0,-l), axis=vec(0,0,l), radius=0.5e-10))  # -z direction (Wall at l,0,-l)

# Initialize list for the walls of the solid
walls = []

# Append walls
walls.append(box(pos=vec(-2*l, 0, 0), size=vec(1e-12, 2*l, 2*l), color=color.green, opacity=0.5))  # Left (-x)
walls.append(box(pos=vec(2*l, 0, 0), size=vec(1e-12, 2*l, 2*l), color=color.green, opacity=0.5))  # Right (+x)
walls.append(box(pos=vec(0, -l, 0), size=vec(4*l, 1e-12, 2*l), color=color.green, opacity=0.5))  # Bottom (-y)
walls.append(box(pos=vec(0, l, 0), size=vec(4*l, 1e-12, 2*l), color=color.green, opacity=0.5))  # Top (+y)
walls.append(box(pos=vec(0, 0, -l), size=vec(4*l, 2*l, 1e-12), color=color.green, opacity=0.5))  # Back (-z)
walls.append(box(pos=vec(0, 0, l), size=vec(4*l, 2*l, 1e-12), color=color.green, opacity=0))   # Front (+z)

# Make buttons
b1 = button(pos=scene.caption_anchor, text='Single Quanta', bind=click1)
b2 = button(pos=scene.caption_anchor, text='Energy of 5 (Al) quanta', bind = click2)

# Initial quanta
Al_quanta = vector(0,0,0) #none in each direction
Al_dq = [0,0,0] #how many to add for this click

#1 Same for Pb
Pb_quanta = vector(0,0,0)
Pb_dq = [0,0,0]

# Labels
Al_label = label(pos=vec(-l,l * 1.2,0), text='Quanta in Al: '+str(Al_quanta), border=1)
Pb_label = label(pos=vec(l,l * 1.2,0), text='Quanta in Pb: '+str(Pb_quanta), border=1)

# Initial energies (ground state)
Al_energy = 0.5 * hbar * sqrt(Al_k / Al_m)   #One half of the ground state energy (for each oscillator)
Pb_energy = 0.5 * hbar * sqrt(Pb_k / Pb_m)

# Initial momenta
Al_p = vec(uniform(-1,1), uniform(-1,1), uniform(-1,1)) * sqrt(2 * Al_m * Al_energy)
Pb_p = vec(uniform(-1,1), uniform(-1,1), uniform(-1,1)) * sqrt(2 * Pb_m * Pb_energy)

# Initialize plot
g1 = graph(width=1000, xmin=0, xmax=200 * max(Al_dE, Pb_dE), ymin=0, ymax=3 * kb * log(100), title='Entropy vs. Energy', xtitle='Energy (J)', ytitle='Entropy (J/K)')  #think about what good values for xmin, xmax, ymin, ymax would be

Al_plot = gcurve(graph=g1, color=color.blue)
Pb_plot = gcurve(graph=g1, color=color.yellow)

while True:
    sleep(1/100.)

    # Reset forces to 0
    Al_F = vector(0,0,0)
    Pb_F = vector(0,0,0)
    
    # Calculate new force on each atom
    #  Determine force from -kx where x is diff b/w new and old position
    #  Remember that the tethered end of the spring is the side away from the atom
    #  and that there is no force for the spring at its "rest" lenth (l)
    #  Total vector force due to all the springs.
    for i in range(6):
        Al_F += -Al_k * (mag(Al.pos - Al_spring[i].pos) - l) * norm(Al.pos - Al_spring[i].pos)
        Pb_F += -Pb_k * (mag(Pb.pos - Pb_spring[i].pos) - l) * norm(Pb.pos - Pb_spring[i].pos)
    
    # Update momentum of atoms
    Al_p += Al_F * dt
    Pb_p += Pb_F * dt

    # Update position of atoms
    Al.pos += (Al_p / Al_m) * dt
    Pb.pos += (Pb_p / Pb_m) * dt

    # Update spring axes (length of springs)
    for i in range(6):
        Al_spring[i].axis = Al.pos - Al_spring[i].pos 
        Pb_spring[i].axis = Pb.pos - Pb_spring[i].pos
        
    # Check to see if button has been pressed
    if Al_dq != [0,0,0]:
        # Update quanta
        Al_quanta += vec(Al_dq[0],Al_dq[1],Al_dq[2])
        Pb_quanta += vec(Pb_dq[0],Pb_dq[1],Pb_dq[2])
        
        # Update energy
        Al_energy += sum(Al_dq) * Al_dE
        Pb_energy += sum(Pb_dq) * Pb_dE

        # Update momentum
        Al_p += Al_F * dt
        Pb_p += Pb_F * dt
        
        # Update labels
        Al_label.text = 'Quanta in Al: '+str(Al_quanta)
        Pb_label.text = 'Quanta in Pb: '+str(Pb_quanta)

        # Calculate total number of quanta
        Al_q = Al_quanta.x + Al_quanta.y + Al_quanta.z
        Pb_q = Pb_quanta.x + Pb_quanta.y + Pb_quanta.z
        
        # Calculate number of microstates in each atom
        Al_Omega = combin(N + Al_q - 1, Al_q) if Al_q > 0 else 1
        Pb_Omega = combin(N + Pb_q - 1, Pb_q) if Pb_q > 0 else 1
        
        # Calculate entropy
        Al_S = kb * log(Al_Omega) if Al_Omega > 1 else 0
        Pb_S = kb * log(Pb_Omega) if Pb_Omega > 1 else 0

        # Plot entropy as a function of energy
        Al_plot.plot(pos=((Al_energy),Al_S))
        Pb_plot.plot(pos=((Pb_energy),Pb_S))

        # Reset dq
        Al_dq = [0,0,0]
        Pb_dq = [0,0,0]

<IPython.core.display.Javascript object>